In [ ]:
import database
import pandas as pd

voos = pd.read_sql( 'select * from voos_sp_sj order by dt_partida_prevista', database.engine )
clima = pd.read_sql()

In [ ]:

voos[ "atrasos_partida" ] = (voos[ "dt_partida_prevista" ] - voos[ "dt_partida_real" ]).dt.seconds / 60
voos[ "atrasos_chegada" ] = (voos[ "dt_chegada_prevista" ] - voos[ "dt_chegada_real" ]).dt.seconds / 60
voos[["atrasos_chegada" , "atrasos_partida"]].min()
# Estaremos preocupados apenas em prever os atrasos de partida.
# Um caso estranho que vemos aqui é que tanto atrasos_chegada e atrasos_partida tem um valor de minimo == 0.
# Não seria de se esperar que houvessem chegadas e partidas adiantadas?

In [ ]:
voos.query("atrasos_partida < 240")[ "atrasos_partida" ].plot( kind = 'hist', bins = 100, normalized = True )

In [ ]:
# E se, ao invés de calcular a quantidade em minutos do atraso,
# Calculássemos a prob. da saída atrasar mais do que N minutos?
# Do histograma, parece haver um threshold forte ao redor
# Dos 20 minutos. Podemos usar 20m como um valor de N.

# Mudança da definição de Regressão para Classificação.

# Iremos predizer a probabilidade de um voos Domesticos regulares de Passageiros ,
# Ter um atraso de saída maior do que 20 minutos.

# Durante a apresentação, podemos ilustrar as justificativas para essa decisão,
# Em particular, o porque de 20 minutos,
# E, porque o atraso da saída ( ao invés do de entrada ).

voos.query( "atrasos_partida < 0" ).count()

In [ ]:
# Consideraremos como baseline um modelo que sempre prevê que o vôo não atrasa.

voos[ "atrasado" ] = voos[ "atrasos_partida" ] >= 20
voos[ "atrasado_previsto" ] = 0

In [ ]:
# Um ponto interessante de se considerar é a metrica utilizada para
# 'scorar' o modelo.
# Acuracia vs F1 vs Precisao são algumas possibilidades,
# E todas tem uma interpretação levemente diferente.

# Se pensarmos nas 4 possibilidades, vulgo:
# Dizemos que atrasa, mas ele não atrasa ( Falso positivo )
# Dizemos que atrasa, e de fato atrasa ( Verdadeiro Positivo)
# não Dizemos que atrasa, e ele atrasa ( Falso negativo)
# não Dizemos que atrasa, e ele não atrasa (Verdadeiro Negativo).

# Vamos pensar na idéia de enviar um ( push | email ) para o cliente, caso acharmos que o voo dele irá atrasar.
# Nesse caso,
# Falsos positivos são PÉSSIMOS. ( 1000x )
# Verdadeiros positivos são Bons. ( 100x )
# Falsos negativos é regular ( 1x )
# Verdadeiro positivo é regular ( 1 x )
from sklearn.metrics import accuracy_score

accuracy_score( voos["atrasado"] , voos["atrasado_previsto"])

In [ ]:
error